In [9]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat
# from langchain.llms.gigachat import GigaChat
from dotenv import load_dotenv
import json
import os 
load_dotenv()
GIGACHAT_CLIENT_SECRET = os.getenv("GIGACHAT_CLIENT_SECRET_B64")

In [2]:
from docx import Document
from docx.shared import Inches

In [3]:
chat = GigaChat(credentials=GIGACHAT_CLIENT_SECRET, verify_ssl_certs=False) #model='GigaChat-Pro'

In [5]:
extraction_prompt = """Определи, есть ли пропуск в строке юридического документа. Если пропуск есть, напиши вопрос пользователю для уточнения данных.

Пример 1:
Строка с пропуском:
Представитель истца: ________________________________
Фрагмент текста:
{{строка с пропуском}}
'(Ф.И.О., адрес)'
Ответ:
Напишите, пожалуйста, ФИО и адрес представителя истца по данному делу

Пример 2:
Строка с пропуском:
Дата судебного заседания: 14 июля 2024 г.
Фрагмент текста:
{{строка с пропуском}}
''
Ответ:
Ничего вставлять не нужно

Пример 3:
Строка с пропуском:
Мировому судье ___________________________
Фрагмент текста:
{{строка с пропуском}}
судебного участка N ______________________

Ответ:
Напишите, пожалуйста, ФИО мирового судьи
"""

In [6]:
messages = [
    SystemMessage(
        content=extraction_prompt
    )
]

In [7]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Open an existing document
doc = Document('iskovoe-zayavlenie-o-rastorzhenii-braka-i-razd.docx')

In [8]:
text_for_llm = []
for p in doc.paragraphs:
    text_for_llm.append(p.text)

text_for_llm.append('') # УБРАТЬ - для индекса контекста на последнем элементе

In [325]:
template = {}
for i in range(len(text_for_llm)):
    if '__' in text_for_llm[i]:
        
        before_context = '\n'.join(text_for_llm[max(0, i-2):i])
        after_context = '\n'.join(text_for_llm[i+1:min(len(text_for_llm), i+3)])
        context = before_context + '\n{{строка с пропуском}}\n' + after_context
        template[str(i)] = {'text': [text_for_llm[i]],
                            'question': None,
                            'field': None,
                            'user_input': None,
                            'final_text': None,
                            'context': context}
 
        

template

{'0': {'text': ['                                 Мировому судье ___________________________'],
  'question': None,
  'field': None,
  'user_input': None,
  'final_text': None,
  'context': '\n{{строка с пропуском}}\n                                 судебного участка N ______________________\n'},
 '1': {'text': ['                                 судебного участка N ______________________'],
  'question': None,
  'field': None,
  'user_input': None,
  'final_text': None,
  'context': '                                 Мировому судье ___________________________\n{{строка с пропуском}}\n\n                                 Истец: ___________________________________'},
 '3': {'text': ['                                 Истец: ___________________________________'],
  'question': None,
  'field': None,
  'user_input': None,
  'final_text': None,
  'context': '                                 судебного участка N ______________________\n\n{{строка с пропуском}}\n                                   

In [330]:
doc_input = f"Строка с пропуском::\n{template['0']['text'][0]}\n\n Фрагмент текста:\n{template['0']['context']}\n\n Ответ:"
print(doc_input)

Строка с пропуском::
                                 Мировому судье ___________________________

 Фрагмент текста:

{{строка с пропуском}}
                                 судебного участка N ______________________


 Ответ:


In [331]:
messages.append(HumanMessage(content=doc_input))
res = chat(messages)
messages.append(res)
print(res.content)

AUTHENTICATION ERROR


Напишите, пожалуйста, ФИО мирового судьи


In [332]:
for key in template:
    doc_input = f"Строка с пропуском::\n{template[key]['text'][0]}\n\n Фрагмент текста:\n{template[key]['context']}\n\n Ответ:"
    messages.append(HumanMessage(content=doc_input))
    res = chat(messages)
    messages.append(res)
    template[key]['question'] = res.content
    print(f"Key: {key}\n\nНужно вставить:\n{res.content}")

Key: 0

Нужно вставить:
Напишите, пожалуйста, ФИО мирового судьи
Key: 1

Нужно вставить:
Напишите, пожалуйста, ФИО мирового судьи
Key: 3

Нужно вставить:
Напишите, пожалуйста, ФИО и адрес истца
Key: 6

Нужно вставить:
Напишите, пожалуйста, ФИО и адрес ответчика
Key: 10

Нужно вставить:
Напишите, пожалуйста, адрес органа опеки и попечительства
Key: 11

Нужно вставить:
Напишите, пожалуйста, адрес органа опеки и попечительства
Key: 13

Нужно вставить:
Ничего вставлять не нужно
Key: 18

Нужно вставить:
Укажите дату вступления в брак с ответчицей(ком).
Key: 21

Нужно вставить:
Укажите имя, число, месяц и год рождения ребенка (детей) от брака.
Key: 22

Нужно вставить:
Укажите имя, число, месяц и год рождения ребенка (детей) от брака.
Key: 25

Нужно вставить:
Укажите причины, по которым совместная жизнь с ответчицей(ком) не сложилась.
Key: 26

Нужно вставить:
Укажите причины, по которым совместная жизнь с ответчицей(ком) не сложилась.
Key: 29

Нужно вставить:
Укажите год и месяц, когда прекра

In [11]:
template2 = template.copy()
for key in template2:
    print(template2[key]['question'])
    answer = input()
    template2[key]['user_input'] = answer

Напишите, пожалуйста, ФИО мирового судьи
Напишите, пожалуйста, ФИО мирового судьи
Напишите, пожалуйста, ФИО и адрес истца
Напишите, пожалуйста, ФИО и адрес ответчика
Напишите, пожалуйста, адрес органа опеки и попечительства
Напишите, пожалуйста, адрес органа опеки и попечительства
Ничего вставлять не нужно
Укажите дату вступления в брак с ответчицей(ком).
Укажите имя, число, месяц и год рождения ребенка (детей) от брака.
Укажите имя, число, месяц и год рождения ребенка (детей) от брака.
Укажите причины, по которым совместная жизнь с ответчицей(ком) не сложилась.
Укажите причины, по которым совместная жизнь с ответчицей(ком) не сложилась.
Укажите год и месяц, когда прекратились брачные отношения между вами.
Укажите, с кем из родителей будет проживать ребенок после развода.
Укажите, с кем из супругов будет проживать ребенок, выплачиваются ли средства на содержание ребенка (детей), добровольно или по судебному решению.
Укажите фамилию, имя и отчество ребенка, с которым ответчик имеет прав

In [12]:
template2

{'0': {'text': ['                                 Мировому судье ___________________________'],
  'question': 'Напишите, пожалуйста, ФИО мирового судьи',
  'field': None,
  'user_input': 'Иванов Андрей Васильевич',
  'final_text': None,
  'context': '\n{{строка с пропуском}}\n                                 судебного участка N ______________________\n'},
 '1': {'text': ['                                 судебного участка N ______________________'],
  'question': 'Напишите, пожалуйста, ФИО мирового судьи',
  'field': None,
  'user_input': 'Иванов Андрей Васильевич',
  'final_text': None,
  'context': '                                 Мировому судье ___________________________\n{{строка с пропуском}}\n\n                                 Истец: ___________________________________'},
 '3': {'text': ['                                 Истец: ___________________________________'],
  'question': 'Напишите, пожалуйста, ФИО и адрес истца',
  'field': None,
  'user_input': 'Андрей',
  'final_text

## Заполнение документа

In [13]:
filling_prompt = """Ты - юрист. Твоя задача заполнять юридические документы, используя данные пользователя.
На вход тебе будет передаваться строка с пропуском и данные пользователя. 
Твоя задача - заменить пропуск в строке, используя переданные данные пользователя. 
Форматирование строки оставляй исходным.

Пример 1:
Строка с пропуском:
                             Представитель истца: ________________________________
Данные пользователя:
Андреев Михаил Сергеевич

Ответ:
                             Представитель истца: Андреев Михаил Сергеевич

Пример 2:
Строка с пропуском:
                             Дата рождения: ________________________________
Данные пользователя:
02.12.2023

Ответ:
                             Дата рождения: 02.12.2023                    
"""

In [14]:

messages = [
    SystemMessage(
        content=filling_prompt
    )
]

In [41]:
s = template2['0']['text'][0]
a = template2['0']['user_input']

In [42]:
user_input = f'Строка с пропуском{s}\nДанные пользователя:{a}\n\nОтвет:'
user_input

'Строка с пропуском                                 Мировому судье ___________________________\nДанные пользователя:Иванов Андрей Васильевич\n\nОтвет:'

In [43]:
messages.append(HumanMessage(content=user_input))
res = chat(messages)
messages.append(res)
print("Bot: ", res.content)

Bot:  Мировому судье Иванову Андрею Васильевичу


In [45]:
import re

# Исходная строка с пропуском
template_str = template2['0']['text'][0]

# Строка с заполнением
fill_str = res.content
fill_str

'Мировому судье Иванову Андрею Васильевичу'

In [34]:
leading_spaces = len(template_str) - len(template_str.lstrip())

# Создание заполненной строки с сохранением отступов
filled_str = ' ' * leading_spaces + fill_str


In [35]:
filled_str

'                                 Ответчик: Курьянов Михаил Павлович ул. Диктора Левитана 55А'

In [51]:
doc = Document('iskovoe-zayavlenie-o-rastorzhenii-braka-i-razd.docx')

In [55]:
paragraph = doc.paragraphs[3]
current_alignment = paragraph.alignment
print(current_alignment)

None


In [48]:
paragraph = doc.paragraphs[0]
current_alignment = paragraph.alignment
print(current_alignment)
paragraph.text = fill_str
paragraph.alignment = current_alignment
doc.save('iskovoe-zayavlenie-o-rastorzhenii-braka-i-razd.docx')